# Task 2. I go right through
Imagine a situation: given cities, between some of which roads are laid, and the lengths of these roads are known. It is necessary to find the optimal distances between all pairs of cities - because sometimes it is more profitable to get from one city to another not directly, but through other cities.

The initial data are specified in the form of a matrix A with dimensions $ N \times N $. The $ A_ {ij} $ element is equal to the length of the road between the city $ i $ and the city $ j $, if it exists, and $ -1 $ otherwise.

As a result, the program should return a matrix of the same form, in which the $ A_ {ij} $ element will contain the length of the optimal path between $ i $ and $ j $.

This is a classic graph theory problem solved by Floyd's algorithm. You need to consistently “allow” to pass through the next city, first through the first, then through the second, and so on. Note the similarities between matrix multiplication and this algorithm: you should get pretty much the same code! The difference is that in matrix multiplication you add the products $ A_ {ik} B_ {kj} $, and in Floyd's algorithm you choose the minimum of $ A_ {ij} $ and $ A_ {ik} + A_ {kj} $.

**Result**: In this problem you need to write a program that accepts a two-dimensional matrix $ A $, in which the element $ A_ {ij} $ contains the distance between the cities $ i $ and $ j $ or $ -1 $, if the direct path from $ i $ does not exist in $ j $. The program should output the matrix by the shortest paths between all vertices in the same form.

# Solution

In [ ]:
def floyd(graph, no_path=-1):
    '''
    Floyd–Warshall algorithm.
    
    The graph defines by a matrix with 
    dimensions NxN. Element graph[i][j] 
    is equal to the length of the road 
    between city i and city j, 
    if it exists, and -1 otherwise (by default).
    
    Parameters
    ----------
    %(input)s
    graph : array-like
        The graph matrix NxN.
    no_path : int, optional
        The no path value between cities.
    
    Returns
    -------
    graph : ndarray
        The shortest path matrix.
    '''
    
    from numpy import array
    from math import inf
    
    graph = array(graph).astype(float)
    graph[graph == no_path] = inf
    
    n = len(graph)
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if graph[i,j] > graph[i,k] + graph[k,j]:
                    graph[i,j] = graph[i,k] + graph[k,j]
                    
    return graph

# Examples

As an example we consider the path matrix

$
A = \begin{pmatrix}
0 & 100 & -1 & 1\\
100 & 0 & 50 & 3\\
-1 & 50 & 0 & 20\\
1 & 3 & 20 & 0
\end{pmatrix}
$.

In [ ]:
A = [[0, 100, -1, 1],[100, 0, 50, 3],[-1, 50, 0, 20],[1, 3, 20, 0]]
A

Apply Floyd's algorithm to the $A$:

In [3]:
floyd(A)

array([[ 0.,  4., 21.,  1.],
       [ 4.,  0., 23.,  3.],
       [21., 23.,  0., 20.],
       [ 1.,  3., 20.,  0.]])

And we get the result:

$
A_{min} = \begin{pmatrix}
0 & 4 & 21 & 1\\
4 & 0 & 23 & 3\\
21 & 23 & 0 & 20\\
1 & 3 & 20 & 0
\end{pmatrix}
$